In [1]:
import pandas as pd
from os import chdir
chdir("C:/Users/manda/Documents/Notes/Datasets/")
from warnings import filterwarnings
filterwarnings("ignore")
import pickle as pkl

In [2]:
loanStatus_model = pkl.load(open("loanStatus_model.pkl", "rb"))
loanAmount_model = pkl.load(open("loanAmount_model.pkl", "rb"))
loanTerm_model = pkl.load(open("loanTerm_model.pkl", "rb"))
clustering_model = pkl.load(open("clustering_model.pkl", "rb"))

## Problem Statement - 1

In [3]:
A = pd.read_csv("loan_training_set.csv")
B = pd.read_csv("loan_testing_set.csv")

In [4]:
# updating cat[] and con[]
cat = []
con = []

for i in A.columns:
    if(A[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [5]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
Q = pd.DataFrame(mm.fit_transform(A[con]), columns=con)

Standardizing training dataset so that sames values of mean and standard deviation will be used for standardizing testing dataset.

## Missing Data Treatment

In [6]:
for i in B.columns:
    if(A[i].dtypes == "object"):
        temp = A[i].mode()[0]
        B[i] = B[i].fillna(temp)
    else:
        temp = A[i].mean()
        B[i] = B[i].fillna(temp)

In [7]:
for i in A.columns:
    if(A[i].dtypes == "object"):
        temp = A[i].mode()[0]
        A[i] = A[i].fillna(temp)
    else:
        temp = A[i].mean()
        A[i] = A[i].fillna(temp)

In [8]:
A.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [9]:
B.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

## Preparing training dataset for predictions - 'Loan_Status'

In [10]:
B1 = B[
    [
        "CoapplicantIncome",
        "ApplicantIncome",
        "LoanAmount",
        "Credit_History",
        "Married",
        "Education",
        "Property_Area",
    ]
]


In [11]:
B1.head()

,CoapplicantIncome,ApplicantIncome,LoanAmount,Credit_History,Married,Education,Property_Area
0,0,5720,110.0,1.000000,Yes,Graduate,Urban
1,1500,3076,126.0,1.000000,Yes,Graduate,Urban
2,1800,5000,208.0,1.000000,Yes,Graduate,Urban
3,2546,2340,100.0,0.842199,Yes,Graduate,Urban
4,0,3276,78.0,1.000000,No,Not Graduate,Urban


In [12]:
# updating cat[] and con[]
cat = []
con = []

for i in B1.columns:
    if(B1[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [13]:
print(cat)

['Married', 'Education', 'Property_Area']


In [14]:
print(con)

['CoapplicantIncome', 'ApplicantIncome', 'LoanAmount', 'Credit_History']


In [15]:
#seperating continous features
X1 = B1[con]

In [16]:
X1.head()

,CoapplicantIncome,ApplicantIncome,LoanAmount,Credit_History
0,0,5720,110.0,1.000000
1,1500,3076,126.0,1.000000
2,1800,5000,208.0,1.000000
3,2546,2340,100.0,0.842199
4,0,3276,78.0,1.000000


In [17]:
# Encoding categorical features
X2 = pd.get_dummies(B1[cat])

In [18]:
X2.head()

,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0,1,1,0,0,0,1
1,0,1,1,0,0,0,1
2,0,1,1,0,0,0,1
3,0,1,1,0,0,0,1
4,1,0,0,1,0,0,1


In [19]:
B1new = X1.join(X2)

In [20]:
B1new.head()

,CoapplicantIncome,ApplicantIncome,LoanAmount,Credit_History,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0,5720,110.0,1.000000,0,1,1,0,0,0,1
1,1500,3076,126.0,1.000000,0,1,1,0,0,0,1
2,1800,5000,208.0,1.000000,0,1,1,0,0,0,1
3,2546,2340,100.0,0.842199,0,1,1,0,0,0,1
4,0,3276,78.0,1.000000,1,0,0,1,0,0,1


In [21]:
B1new.shape

(367, 11)

In [22]:
# Making Predictions
loanStatus_predictions = loanStatus_model.predict(B1new)

In [23]:
print(loanStatus_predictions)

['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N'
 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'Y'
 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y'
 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N'
 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'N' 'Y'
 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y'
 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y

In [24]:
B['pred_LoanStatus'] = loanStatus_predictions

In [25]:
B.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,pred_LoanStatus
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.000000,Urban,Y
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.000000,Urban,Y
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.000000,Urban,Y
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,0.842199,Urban,Y
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.000000,Urban,Y


In [26]:
B[B["pred_LoanStatus"] == "N"].head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,pred_LoanStatus
7,LP001056,Male,Yes,2,Not Graduate,No,3881,0,147.0,360.0,0.0,Rural,N
13,LP001094,Male,Yes,2,Graduate,No,12173,0,166.0,360.0,0.0,Semiurban,N
35,LP001203,Male,No,0,Graduate,No,3150,0,176.0,360.0,0.0,Semiurban,N
55,LP001313,Male,No,0,Graduate,No,2750,0,130.0,360.0,0.0,Urban,N
58,LP001323,Female,Yes,2,Graduate,No,2779,3664,176.0,360.0,0.0,Semiurban,N


In [27]:
B[B["pred_LoanStatus"] == "N"].index

Int64Index([  7,  13,  35,  55,  58,  63,  66,  67,  69,  80,  82,  84,  89,
             94, 101, 106, 117, 118, 119, 123, 124, 126, 140, 142, 147, 153,
            160, 161, 165, 166, 168, 173, 174, 192, 193, 196, 198, 211, 224,
            229, 235, 236, 241, 243, 245, 250, 255, 266, 268, 271, 273, 274,
            278, 293, 301, 311, 317, 325, 339, 346, 354],
           dtype='int64')

These are all the indexes for which the model has predicted 'Loan_Status' as 'N'.

## Preparing training dataset for predictions - 'Loan_Amount'

In [28]:
B2 = B[[
        "CoapplicantIncome",
        "ApplicantIncome",
        "Married",
        "Education",
        "Gender",
        "Dependents",
        "Self_Employed"
]]

In [29]:
B2.head()

,CoapplicantIncome,ApplicantIncome,Married,Education,Gender,Dependents,Self_Employed
0,0,5720,Yes,Graduate,Male,0,No
1,1500,3076,Yes,Graduate,Male,1,No
2,1800,5000,Yes,Graduate,Male,2,No
3,2546,2340,Yes,Graduate,Male,2,No
4,0,3276,No,Not Graduate,Male,0,No


In [30]:
B2.shape

(367, 7)

In [31]:
B2['Dependents'].unique()

array(['0', '1', '2', '3+'], dtype=object)

In [32]:
B2['Self_Employed'].unique()

array(['No', 'Yes'], dtype=object)

In [33]:
# updating cat[] and con[]
cat = []
con = []

for i in B2.columns:
    if(B2[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [34]:
print(cat)

['Married', 'Education', 'Gender', 'Dependents', 'Self_Employed']


In [35]:
print(con)

['CoapplicantIncome', 'ApplicantIncome']


In [36]:
#standardizing continous features
X1 = pd.DataFrame(mm.fit_transform(B2[con]), columns=con)

In [37]:
X1.head()

,CoapplicantIncome,ApplicantIncome
0,0.000000,0.078865
1,0.062500,0.042411
2,0.075000,0.068938
3,0.106083,0.032263
4,0.000000,0.045168


In [38]:
#encoding categorical features
X2 = pd.get_dummies(B2[cat])

In [39]:
X2.head()

,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Gender_Female,Gender_Male,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Self_Employed_No,Self_Employed_Yes
0,0,1,1,0,0,1,1,0,0,0,1,0
1,0,1,1,0,0,1,0,1,0,0,1,0
2,0,1,1,0,0,1,0,0,1,0,1,0
3,0,1,1,0,0,1,0,0,1,0,1,0
4,1,0,0,1,0,1,1,0,0,0,1,0


In [40]:
B2new = X1.join(X2)

In [41]:
B2new.head()

,CoapplicantIncome,ApplicantIncome,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Gender_Female,Gender_Male,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Self_Employed_No,Self_Employed_Yes
0,0.000000,0.078865,0,1,1,0,0,1,1,0,0,0,1,0
1,0.062500,0.042411,0,1,1,0,0,1,0,1,0,0,1,0
2,0.075000,0.068938,0,1,1,0,0,1,0,0,1,0,1,0
3,0.106083,0.032263,0,1,1,0,0,1,0,0,1,0,1,0
4,0.000000,0.045168,1,0,0,1,0,1,1,0,0,0,1,0


In [42]:
loanAmount_predictions = loanAmount_model.predict(B2new)

In [43]:
print(loanAmount_predictions)

[[ 87.8119253 ]
 [ 96.38956335]
 [104.20413162]
 [100.77653269]
 [ 70.63811843]
 [ 89.1745846 ]
 [ 69.5528686 ]
 [ 84.40175636]
 [118.56626454]
 [ 80.73956412]
 [ 70.12927062]
 [ 94.0222483 ]
 [ 97.51200354]
 [114.55049259]
 [ 75.57479269]
 [ 92.58670259]
 [108.87473375]
 [101.3128539 ]
 [138.72042674]
 [ 90.63311467]
 [ 86.40612869]
 [ 78.21338757]
 [ 79.72055509]
 [ 65.75521433]
 [109.76416476]
 [194.07782682]
 [ 99.58496154]
 [112.25616153]
 [ 86.79622327]
 [ 89.47787084]
 [ 62.21528385]
 [102.53367142]
 [ 86.730741  ]
 [ 93.8500979 ]
 [ 87.64839573]
 [ 77.63290378]
 [122.28738074]
 [ 93.51237693]
 [ 85.65624242]
 [ 89.22179531]
 [ 91.82576195]
 [ 72.76375233]
 [ 80.10184072]
 [100.40065963]
 [ 72.82426396]
 [104.12638023]
 [ 83.87699616]
 [ 78.02252726]
 [105.05091849]
 [105.77323113]
 [ 95.66153186]
 [ 95.79129006]
 [105.77649722]
 [ 97.60236127]
 [ 79.93976761]
 [ 76.53269229]
 [ 67.50730114]
 [109.45434431]
 [101.58410053]
 [102.14595913]
 [ 80.35144009]
 [ 93.54244116]
 [ 85.39

In [44]:
B["pred_LoanAmount"] = loanAmount_predictions

In [45]:
B.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,pred_LoanStatus,pred_LoanAmount
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.000000,Urban,Y,87.811925
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.000000,Urban,Y,96.389563
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.000000,Urban,Y,104.204132
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,0.842199,Urban,Y,100.776533
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.000000,Urban,Y,70.638118


## Problem Statement 3 - predicting 'Loan_AmountTerm'

### Preparing training dataset for predictions - 'Loan_AmountTerm'

In [46]:
B3 = B[["Gender", "Married", "Dependents", "Education", "Self_Employed", "CoapplicantIncome", "ApplicantIncome"]]

In [47]:
B3.head()

,Gender,Married,Dependents,Education,Self_Employed,CoapplicantIncome,ApplicantIncome
0,Male,Yes,0,Graduate,No,0,5720
1,Male,Yes,1,Graduate,No,1500,3076
2,Male,Yes,2,Graduate,No,1800,5000
3,Male,Yes,2,Graduate,No,2546,2340
4,Male,No,0,Not Graduate,No,0,3276


In [48]:
B3.shape

(367, 7)

In [49]:
# updating cat[] and con[]
cat = []
con = []

for i in B3.columns:
    if(B3[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [50]:
print(cat)

['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed']


In [51]:
print(con)

['CoapplicantIncome', 'ApplicantIncome']


In [52]:
#standardizing continous features
X1 = pd.DataFrame(mm.transform(B3[con]), columns=con)

In [53]:
X1.head()

,CoapplicantIncome,ApplicantIncome
0,0.000000,0.078865
1,0.062500,0.042411
2,0.075000,0.068938
3,0.106083,0.032263
4,0.000000,0.045168


In [54]:
#encoding categorical features
X2 = pd.get_dummies(B3[cat])

In [55]:
X2.head()

,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes
0,0,1,0,1,1,0,0,0,1,0,1,0
1,0,1,0,1,0,1,0,0,1,0,1,0
2,0,1,0,1,0,0,1,0,1,0,1,0
3,0,1,0,1,0,0,1,0,1,0,1,0
4,0,1,1,0,1,0,0,0,0,1,1,0


In [56]:
B3new = X1.join(X2)

In [57]:
B3new.head()

,CoapplicantIncome,ApplicantIncome,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes
0,0.000000,0.078865,0,1,0,1,1,0,0,0,1,0,1,0
1,0.062500,0.042411,0,1,0,1,0,1,0,0,1,0,1,0
2,0.075000,0.068938,0,1,0,1,0,0,1,0,1,0,1,0
3,0.106083,0.032263,0,1,0,1,0,0,1,0,1,0,1,0
4,0.000000,0.045168,0,1,1,0,1,0,0,0,0,1,1,0


In [58]:
import numpy as np
loanTerm_predictions = loanTerm_model.predict(B3new)
loanTerm_predictions = np.round(loanTerm_predictions).astype(int)

In [59]:
print(loanTerm_predictions)

[[355]
 [342]
 [356]
 [356]
 [352]
 [340]
 [347]
 [345]
 [354]
 [350]
 [352]
 [343]
 [354]
 [354]
 [371]
 [351]
 [355]
 [346]
 [349]
 [363]
 [330]
 [349]
 [364]
 [358]
 [352]
 [350]
 [356]
 [352]
 [355]
 [363]
 [359]
 [356]
 [363]
 [354]
 [355]
 [364]
 [353]
 [352]
 [362]
 [363]
 [354]
 [371]
 [342]
 [361]
 [371]
 [353]
 [342]
 [351]
 [361]
 [330]
 [342]
 [333]
 [341]
 [356]
 [341]
 [364]
 [358]
 [355]
 [362]
 [346]
 [342]
 [352]
 [372]
 [371]
 [333]
 [353]
 [343]
 [356]
 [354]
 [353]
 [371]
 [364]
 [342]
 [352]
 [340]
 [362]
 [354]
 [361]
 [341]
 [361]
 [345]
 [322]
 [344]
 [354]
 [355]
 [352]
 [354]
 [364]
 [352]
 [370]
 [352]
 [343]
 [341]
 [343]
 [341]
 [352]
 [340]
 [353]
 [344]
 [345]
 [364]
 [363]
 [343]
 [345]
 [363]
 [362]
 [364]
 [346]
 [355]
 [342]
 [369]
 [355]
 [371]
 [362]
 [352]
 [352]
 [359]
 [342]
 [370]
 [356]
 [364]
 [353]
 [345]
 [360]
 [368]
 [364]
 [363]
 [356]
 [364]
 [329]
 [355]
 [364]
 [341]
 [359]
 [364]
 [355]
 [357]
 [342]
 [363]
 [361]
 [364]
 [344]
 [332]

In [60]:
B['pred_LoanTerm'] = loanTerm_predictions

If duration is less than 20 years, is customer eligible for required amount for some longer duration? What is that duration?(Regression)

In [61]:
B[(B["Loan_Amount_Term"] < 360) & (B["pred_LoanStatus"] == 'N')]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,pred_LoanStatus,pred_LoanAmount,pred_LoanTerm
84,LP001450,Male,Yes,0,Graduate,No,4456,0,131.0,180.0,0.0,Semiurban,N,84.335257,355
117,LP001611,Male,Yes,1,Graduate,No,1516,2900,80.0,342.0,0.0,Rural,N,99.396769,342
173,LP001979,Male,No,0,Graduate,No,3017,2845,159.0,180.0,0.0,Urban,N,92.097723,363
245,LP002355,Male,Yes,0,Graduate,No,3186,3145,150.0,180.0,0.0,Semiurban,N,97.236589,354
325,LP002802,Male,No,0,Graduate,No,2875,2416,95.0,6.0,0.0,Semiurban,N,89.470823,363
354,LP002921,Male,Yes,3+,Not Graduate,No,5316,187,158.0,180.0,0.0,Semiurban,N,97.418731,333


## Clustering

In [62]:
A.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.00000,0.0,146.412162,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,5405.54085,1508.0,128.000000,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.00000,0.0,66.000000,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.00000,2358.0,120.000000,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.00000,0.0,141.000000,360.0,1.0,Urban,Y


In [63]:
B.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,pred_LoanStatus,pred_LoanAmount,pred_LoanTerm
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.000000,Urban,Y,87.811925,355
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.000000,Urban,Y,96.389563,342
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.000000,Urban,Y,104.204132,356
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,0.842199,Urban,Y,100.776533,356
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.000000,Urban,Y,70.638118,352


In [64]:
C = B[["ApplicantIncome", "LoanAmount"]]

In [65]:
C.head()

,ApplicantIncome,LoanAmount
0,5720,110.0
1,3076,126.0
2,5000,208.0
3,2340,100.0
4,3276,78.0


In [66]:
# updating cat[] and con[]
cat = []
con = []

for i in C.columns:
    if(C[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [67]:
print(cat)

[]


In [68]:
print(con)

['ApplicantIncome', 'LoanAmount']


In [69]:
mm = MinMaxScaler()
Q = pd.DataFrame(mm.fit_transform(A[con]), columns=con)

In [70]:
Cnew = pd.DataFrame(mm.transform(C), columns=C.columns)

In [71]:
Cnew.head()

,ApplicantIncome,LoanAmount
0,0.068893,0.146165
1,0.036190,0.169320
2,0.059988,0.287988
3,0.027087,0.131693
4,0.038664,0.099855


In [72]:
clustering_model.cluster_centers_

array([[0.2515308 , 0.31093882],
       [0.24513674, 0.71904233],
       [0.75539575, 0.2472002 ],
       [0.14486962, 0.39544323],
       [0.14291491, 0.14631793],
       [0.44428774, 0.44220711],
       [0.27018245, 0.47574124],
       [0.45839571, 0.71710949],
       [0.75768875, 0.64386792],
       [0.13164995, 0.27153404]])

In [73]:
label_pred = clustering_model.predict(Cnew)

In [74]:
label_pred

array([4, 4, 9, 4, 4, 9, 4, 4, 3, 4, 4, 9, 4, 9, 4, 4, 9, 4, 3, 4, 4, 4,
       4, 4, 3, 9, 4, 3, 4, 4, 4, 9, 9, 9, 4, 9, 9, 4, 4, 4, 4, 4, 4, 9,
       4, 9, 9, 4, 9, 9, 4, 4, 9, 4, 4, 4, 4, 4, 9, 4, 4, 4, 9, 4, 4, 4,
       4, 4, 9, 9, 4, 4, 4, 9, 4, 9, 9, 4, 4, 9, 9, 8, 9, 3, 4, 4, 4, 4,
       4, 9, 4, 6, 9, 4, 4, 4, 3, 4, 4, 9, 4, 4, 4, 4, 4, 9, 4, 4, 4, 9,
       4, 4, 4, 4, 9, 9, 9, 4, 4, 9, 4, 4, 4, 9, 3, 4, 4, 9, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 9, 4, 4, 7, 3, 4, 4, 9, 4, 9, 9, 4, 4, 4,
       4, 4, 9, 4, 9, 4, 9, 4, 4, 4, 4, 9, 4, 4, 4, 4, 4, 4, 4, 9, 4, 9,
       4, 4, 4, 9, 9, 9, 4, 9, 4, 4, 9, 9, 4, 3, 4, 4, 9, 4, 6, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 9, 4, 4, 4, 4, 9, 9, 9, 4, 9, 4, 9, 9, 4,
       4, 9, 4, 9, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 9, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 9, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 9, 9, 4, 9, 4, 4, 0,
       9, 4, 4, 4, 4, 4, 4, 9, 0, 4, 4, 4, 4, 4, 9, 0, 4, 4, 4, 4, 1, 3,
       4, 4, 4, 4, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [75]:
B['Cluster'] = label_pred

If duration is less than equal to 20 years, is customer eligible for required amount for some longer duration? What is that duration?

If duration is less than equal to 20 years, is customer eligible for required amount for some longer duration? What is that duration?

In [76]:
B[(B["Loan_Amount_Term"] < 360) & (B["pred_LoanStatus"] == 'N')]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,pred_LoanStatus,pred_LoanAmount,pred_LoanTerm,Cluster
84,LP001450,Male,Yes,0,Graduate,No,4456,0,131.0,180.0,0.0,Semiurban,N,84.335257,355,4
117,LP001611,Male,Yes,1,Graduate,No,1516,2900,80.0,342.0,0.0,Rural,N,99.396769,342,4
173,LP001979,Male,No,0,Graduate,No,3017,2845,159.0,180.0,0.0,Urban,N,92.097723,363,9
245,LP002355,Male,Yes,0,Graduate,No,3186,3145,150.0,180.0,0.0,Semiurban,N,97.236589,354,9
325,LP002802,Male,No,0,Graduate,No,2875,2416,95.0,6.0,0.0,Semiurban,N,89.470823,363,4
354,LP002921,Male,Yes,3+,Not Graduate,No,5316,187,158.0,180.0,0.0,Semiurban,N,97.418731,333,9
